In [1]:
# Nuestro path de inicio siempre debe ser el home dir, e
import os
os.chdir(os.path.abspath(".."))

# Imports
import json
import pandas as pd
import stable_whisper
import warnings
from src.votaciones import votacion_promedio_simple

# ignoramos warnings
warnings.filterwarnings('ignore')

In [2]:
# Archivos importantes y paths
df_annotations = pd.read_excel('data/annotations.xlsx')

# Omitimos todos los audios en development
df_annotations = df_annotations[df_annotations['Type'] != 'Development'].reset_index(drop = True)

In [3]:
# Obtenemos el json con los intervalos
with open('data/intervalos_transcripciones.json', 'r') as f:
    intervalos_transcripciones = json.load(f)
    
# Comprabamos seguimos teniendo los 244 audios
len(intervalos_transcripciones.keys())

244

In [4]:
# Audios que no estan en development
lista_audios = df_annotations['Audio_Name'].unique()
len(lista_audios)

205

In [11]:
# Creo mi diccionario que voy a guardare como json
targets_mean_vote = {}

# Loop por cada audio en la lista de audios que no estan en development
for audio_name in lista_audios:
    
    print('Procesando el audio:', audio_name)
    
    df_audio = df_annotations[df_annotations['Audio_Name'] == audio_name]
    partes = df_audio['Part_Num'].max()
    
    # Inicializo valores para el audio en el diccionario
    targets_mean_vote[audio_name] = {}
    targets_mean_vote[audio_name]['rangos'] = []
    targets_mean_vote[audio_name]['targets'] = []
    
    # Loop por cada parte del audio
    for part_num in range(partes):
        part_num = part_num + 1
        
        print(f'Procesando parte {part_num} de {audio_name}')
        
        # Obtengo votación para la parte del audio
        df_votacion = votacion_promedio_simple(df_annotations, audio_name = audio_name, part_num = part_num)
        
        # Verificamos que solo halla un start time y nos lo quedamos
        assert len(df_audio[df_audio['Part_Num'] == part_num]['start_time'].unique()) == 1, 'Más de un start time'
        start_time = df_audio[df_audio['Part_Num'] == part_num]['start_time'].unique()[0]
        
        # Agregamos al audio el start time necesario
        df_votacion['Time'] = df_votacion['Time'] + start_time
        
        # Loop principal para cargar las votaciones promedio
        i = 0
        
        int_act = intervalos_transcripciones[audio_name] 
        ts_primera_anotacion = df_votacion['Time'][0]
        ts_ultima_anotacion = df_votacion['Time'][len(df_votacion) - 1]
        
        while i + 1 <= len(int_act) and int_act[i][1] < ts_ultima_anotacion:
            
            if int_act[i][0] > ts_primera_anotacion:
                inicio = intervalos_transcripciones[audio_name][i][0]
                fin = intervalos_transcripciones[audio_name][i][1]
                
                votacion_promedio = df_votacion[(df_votacion['Time'] >= inicio) & (df_votacion['Time'] <= fin)][['Valence','Arousal','Dominance']].mean().values.tolist()
                
                targets_mean_vote[audio_name]['targets'].append(votacion_promedio)
                targets_mean_vote[audio_name]['rangos'].append((inicio, fin))
            
            i += 1
            
    assert len(targets_mean_vote[audio_name]['targets']) == len(targets_mean_vote[audio_name]['rangos']), 'Rangos no coinciden'
        
    print(f'{audio_name} procesado con éxito')
    print('')
    print('---------------------------------------------------------')

Procesando el audio: MSP-Conversation_0021.wav
Procesando parte 1 de MSP-Conversation_0021.wav
Procesando parte 2 de MSP-Conversation_0021.wav
Procesando parte 3 de MSP-Conversation_0021.wav
Procesando parte 4 de MSP-Conversation_0021.wav
MSP-Conversation_0021.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0023.wav
Procesando parte 1 de MSP-Conversation_0023.wav
Procesando parte 2 de MSP-Conversation_0023.wav
MSP-Conversation_0023.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0035.wav
Procesando parte 1 de MSP-Conversation_0035.wav
Procesando parte 2 de MSP-Conversation_0035.wav
MSP-Conversation_0035.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0047.wav
Procesando parte 1 de MSP-Conversation_0047.wav
Procesando parte 2 de MSP-Conversation_0047.wav
MSP-Conversation_

Procesando parte 3 de MSP-Conversation_0361.wav
MSP-Conversation_0361.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0380.wav
Procesando parte 1 de MSP-Conversation_0380.wav
Procesando parte 2 de MSP-Conversation_0380.wav
MSP-Conversation_0380.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0381.wav
Procesando parte 1 de MSP-Conversation_0381.wav
Procesando parte 2 de MSP-Conversation_0381.wav
MSP-Conversation_0381.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0418.wav
Procesando parte 1 de MSP-Conversation_0418.wav
Procesando parte 2 de MSP-Conversation_0418.wav
MSP-Conversation_0418.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0422.wav
Procesando parte 1 de MSP-Conversation_0422.wav
Procesan

Procesando parte 2 de MSP-Conversation_0228.wav
Procesando parte 3 de MSP-Conversation_0228.wav
Procesando parte 4 de MSP-Conversation_0228.wav
MSP-Conversation_0228.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0340.wav
Procesando parte 1 de MSP-Conversation_0340.wav
Procesando parte 2 de MSP-Conversation_0340.wav
Procesando parte 3 de MSP-Conversation_0340.wav
Procesando parte 4 de MSP-Conversation_0340.wav
MSP-Conversation_0340.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0593.wav
Procesando parte 1 de MSP-Conversation_0593.wav
Procesando parte 2 de MSP-Conversation_0593.wav
MSP-Conversation_0593.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0642.wav
Procesando parte 1 de MSP-Conversation_0642.wav
Procesando parte 2 de MSP-Conversation_0642.wav
Procesando parte

Procesando parte 2 de MSP-Conversation_0055.wav
MSP-Conversation_0055.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0053.wav
Procesando parte 1 de MSP-Conversation_0053.wav
Procesando parte 2 de MSP-Conversation_0053.wav
Procesando parte 3 de MSP-Conversation_0053.wav
MSP-Conversation_0053.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0101.wav
Procesando parte 1 de MSP-Conversation_0101.wav
Procesando parte 2 de MSP-Conversation_0101.wav
Procesando parte 3 de MSP-Conversation_0101.wav
Procesando parte 4 de MSP-Conversation_0101.wav
MSP-Conversation_0101.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0110.wav
Procesando parte 1 de MSP-Conversation_0110.wav
Procesando parte 2 de MSP-Conversation_0110.wav
Procesando parte 3 de MSP-Conversation_0110.wav
Procesando parte

Procesando parte 2 de MSP-Conversation_0700.wav
Procesando parte 3 de MSP-Conversation_0700.wav
Procesando parte 4 de MSP-Conversation_0700.wav
MSP-Conversation_0700.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0706.wav
Procesando parte 1 de MSP-Conversation_0706.wav
Procesando parte 2 de MSP-Conversation_0706.wav
Procesando parte 3 de MSP-Conversation_0706.wav
MSP-Conversation_0706.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0715.wav
Procesando parte 1 de MSP-Conversation_0715.wav
Procesando parte 2 de MSP-Conversation_0715.wav
Procesando parte 3 de MSP-Conversation_0715.wav
Procesando parte 4 de MSP-Conversation_0715.wav
MSP-Conversation_0715.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0782.wav
Procesando parte 1 de MSP-Conversation_0782.wav
Procesando parte

Procesando parte 2 de MSP-Conversation_1701.wav
Procesando parte 3 de MSP-Conversation_1701.wav
MSP-Conversation_1701.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_1710.wav
Procesando parte 1 de MSP-Conversation_1710.wav
Procesando parte 2 de MSP-Conversation_1710.wav
Procesando parte 3 de MSP-Conversation_1710.wav
MSP-Conversation_1710.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_1896.wav
Procesando parte 1 de MSP-Conversation_1896.wav
Procesando parte 2 de MSP-Conversation_1896.wav
Procesando parte 3 de MSP-Conversation_1896.wav
MSP-Conversation_1896.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_1968.wav
Procesando parte 1 de MSP-Conversation_1968.wav
Procesando parte 2 de MSP-Conversation_1968.wav
Procesando parte 3 de MSP-Conversation_1968.wav
MSP-Conversation

MSP-Conversation_0686.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0692.wav
Procesando parte 1 de MSP-Conversation_0692.wav
Procesando parte 2 de MSP-Conversation_0692.wav
Procesando parte 3 de MSP-Conversation_0692.wav
MSP-Conversation_0692.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0714.wav
Procesando parte 1 de MSP-Conversation_0714.wav
Procesando parte 2 de MSP-Conversation_0714.wav
MSP-Conversation_0714.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0054.wav
Procesando parte 1 de MSP-Conversation_0054.wav
Procesando parte 2 de MSP-Conversation_0054.wav
Procesando parte 3 de MSP-Conversation_0054.wav
MSP-Conversation_0054.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0197.wav
Procesan

Procesando parte 2 de MSP-Conversation_1452.wav
MSP-Conversation_1452.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_1464.wav
Procesando parte 1 de MSP-Conversation_1464.wav
Procesando parte 2 de MSP-Conversation_1464.wav
Procesando parte 3 de MSP-Conversation_1464.wav
Procesando parte 4 de MSP-Conversation_1464.wav
MSP-Conversation_1464.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_1512.wav
Procesando parte 1 de MSP-Conversation_1512.wav
Procesando parte 2 de MSP-Conversation_1512.wav
Procesando parte 3 de MSP-Conversation_1512.wav
Procesando parte 4 de MSP-Conversation_1512.wav
MSP-Conversation_1512.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_1523.wav
Procesando parte 1 de MSP-Conversation_1523.wav
Procesando parte 2 de MSP-Conversation_1523.wav
Procesando parte

Validaciones

In [15]:
print(len(targets_mean_vote.keys()))
print(targets_mean_vote.keys())

205
dict_keys(['MSP-Conversation_0021.wav', 'MSP-Conversation_0023.wav', 'MSP-Conversation_0035.wav', 'MSP-Conversation_0047.wav', 'MSP-Conversation_0061.wav', 'MSP-Conversation_0079.wav', 'MSP-Conversation_0083.wav', 'MSP-Conversation_0087.wav', 'MSP-Conversation_0088.wav', 'MSP-Conversation_0103.wav', 'MSP-Conversation_0114.wav', 'MSP-Conversation_0125.wav', 'MSP-Conversation_0135.wav', 'MSP-Conversation_0147.wav', 'MSP-Conversation_0153.wav', 'MSP-Conversation_0160.wav', 'MSP-Conversation_0166.wav', 'MSP-Conversation_0167.wav', 'MSP-Conversation_0172.wav', 'MSP-Conversation_0180.wav', 'MSP-Conversation_0190.wav', 'MSP-Conversation_0201.wav', 'MSP-Conversation_0202.wav', 'MSP-Conversation_0218.wav', 'MSP-Conversation_0227.wav', 'MSP-Conversation_0231.wav', 'MSP-Conversation_0257.wav', 'MSP-Conversation_0269.wav', 'MSP-Conversation_0281.wav', 'MSP-Conversation_0295.wav', 'MSP-Conversation_0300.wav', 'MSP-Conversation_0361.wav', 'MSP-Conversation_0380.wav', 'MSP-Conversation_0381.wav',

In [16]:
for i in targets_mean_vote.keys():
    print(i, len(targets_mean_vote[i]['rangos']))

MSP-Conversation_0021.wav 474
MSP-Conversation_0023.wav 219
MSP-Conversation_0035.wav 211
MSP-Conversation_0047.wav 242
MSP-Conversation_0061.wav 294
MSP-Conversation_0079.wav 447
MSP-Conversation_0083.wav 272
MSP-Conversation_0087.wav 580
MSP-Conversation_0088.wav 304
MSP-Conversation_0103.wav 758
MSP-Conversation_0114.wav 290
MSP-Conversation_0125.wav 567
MSP-Conversation_0135.wav 189
MSP-Conversation_0147.wav 287
MSP-Conversation_0153.wav 135
MSP-Conversation_0160.wav 452
MSP-Conversation_0166.wav 248
MSP-Conversation_0167.wav 324
MSP-Conversation_0172.wav 80
MSP-Conversation_0180.wav 220
MSP-Conversation_0190.wav 363
MSP-Conversation_0201.wav 326
MSP-Conversation_0202.wav 211
MSP-Conversation_0218.wav 569
MSP-Conversation_0227.wav 341
MSP-Conversation_0231.wav 369
MSP-Conversation_0257.wav 236
MSP-Conversation_0269.wav 452
MSP-Conversation_0281.wav 374
MSP-Conversation_0295.wav 178
MSP-Conversation_0300.wav 426
MSP-Conversation_0361.wav 478
MSP-Conversation_0380.wav 245
MSP-Convers

In [18]:
# Guardo el diccionario en archivo json
with open('data/objetivos_por_voto_promedio.json', 'w') as f:
    json.dump(targets_mean_vote, f)